In [1]:
import msprime
from trails.cutpoints import cutpoints_ABC, cutpoints_AB
import re
from ray import shutdown
# %load_ext rpy2.ipython

In [5]:
import pandas as pd
import numpy as np

n_int_AB = 1
n_int_ABC = 1

mu = 1.25e-8
g = 29
N_AB = 7_000*2*mu
N_BC = 7_000*2*mu
N_ABC = 7_000*2*mu
N_ref = N_ABC
t_1 = 62_041/g*mu
# t_2 = (580_000-t_1)/g*mu
# t_3 = (6_000_000-t_2)/g*mu
t_2 = 580_000/g*mu-t_1
t_3 = 6_000_000/g*mu-t_2
t_A = t_1
t_B = t_1
t_C = t_1+t_2
t_m = t_1-54_000/g*mu
t_upper = t_3-cutpoints_ABC(n_int_ABC,  1/N_ABC)[-2]
t_out = t_1+t_2+t_3+2*N_ABC
r = 1e-8/mu
mu = mu/mu
m = 0.02

seed = 2

from trails.optimizer import trans_emiss_calc_introgression

transitions, emissions, starting, hidden_states, observed_states = trans_emiss_calc_introgression(
    t_1, t_1, t_C, t_2, t_upper, t_out, t_m,
    N_AB, N_BC, N_ABC, r, m, n_int_AB, n_int_ABC)

# shutdown()


Initializing...
49d21daec01431ca5ee7809635890e
Initializing...
5ad5364cbf82a51581d996c1b2a30b


In [6]:
m*(2/3)*np.exp(-(t_2+t_m)/(N_AB))+(1-m)*(2/3)*np.exp(-(t_2)/(N_BC))

0.186072577384884

In [7]:
acc = 0
for i in hidden_states:
    if hidden_states[i][0] in [2, 3]:
        acc += starting[i]
acc

0.18607257738488395

In [164]:
pd.DataFrame(transitions, columns=[str(i) for i in hidden_states.values()]).to_csv(f'tables/transitions_{n_int_ABC}.csv', index = False)

In [165]:
n_sites = 1_000_000

demography = msprime.Demography()
demography.add_population(name="A", initial_size=N_AB, default_sampling_time=t_1-t_A)
demography.add_population(name="B", initial_size=N_AB, default_sampling_time=t_1-t_B)
demography.add_population(name="B_anc", initial_size=N_AB, initially_active=False)
demography.add_population(name="C", initial_size=N_BC, default_sampling_time=t_1+t_2-t_C)
demography.add_population(name="D", initial_size=N_AB, default_sampling_time=t_1-t_1)
demography.add_population(name="AB", initial_size=N_AB)
demography.add_population(name="ABC", initial_size=N_ABC)
demography.add_population(name="ABCD", initial_size=N_ABC)
demography.add_admixture(time = t_1-t_m, derived="B", ancestral=["B_anc", "C"], proportions=(1-m, m))
demography.add_population_split(time=t_1, derived=["A", "B_anc"], ancestral="AB")
demography.add_population_split(time=t_1+t_2, derived=["AB", "C"], ancestral="ABC")
demography.add_population_split(time=t_1+t_2+t_3, derived=["ABC", "D"], ancestral="ABCD")

# demography.debug()

ts = msprime.sim_ancestry(
    {"A": 1, "B": 1, "C": 1,
     "D": 1
    },
    demography=demography,
    recombination_rate=r,
    sequence_length=n_sites,
    ploidy=1,
    random_seed=seed
)



In [166]:
# Add mutations
dct = {v: k for k, v in observed_states.items()}

mutated_ts = msprime.sim_mutations(ts, rate=mu, random_seed=seed)

nochange_lst = [dct['AAAA'], dct['CCCC'], dct['TTTT'], dct['GGGG']]
sim_genome = np.random.choice(nochange_lst, n_sites)

mut_lst = []
mut_loc = []
for variant in mutated_ts.variants():
    mut_loc.append(variant.site.position)
    mut_lst.append(''.join([variant.alleles[i] for i in variant.genotypes]))

for i in range(len(mut_loc)):
    sim_genome[int(mut_loc[i])] = dct[mut_lst[i]]

E = sim_genome

In [167]:
# from trails.optimizer import loglik_wrapper

# loglik = loglik_wrapper(transitions, emissions, starting, [E])

# loglik

In [168]:
from trails.optimizer import post_prob_wrapper

postprobs = post_prob_wrapper(transitions, emissions, starting, [E])


In [169]:
postprobs_df = pd.DataFrame(postprobs[0], columns=[str(i) for i in hidden_states.values()])
postprobs_df = postprobs_df[0:1000000]

In [170]:
postprobs_df

,"(0, 0, 0)","(0, 0, 1)","(0, 0, 2)","(0, 0, 3)","(0, 0, 4)","(0, 0, 5)","(0, 0, 6)","(1, 0, 0)","(1, 0, 1)","(1, 0, 2)",...,"(3, 5, 5)","(3, 5, 6)","(3, 6, 6)","(4, 0, 0)","(4, 0, 1)","(4, 0, 2)","(4, 0, 3)","(4, 0, 4)","(4, 0, 5)","(4, 0, 6)"
0,0.124384,0.142725,0.158620,0.166931,0.162511,0.142330,0.098478,0.000022,0.000101,0.000122,...,0.000001,0.000003,1.236673e-07,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002
1,0.124381,0.142725,0.158624,0.166938,0.162518,0.142332,0.098463,0.000022,0.000101,0.000122,...,0.000001,0.000003,1.235446e-07,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002
2,0.124378,0.142726,0.158629,0.166945,0.162525,0.142334,0.098448,0.000022,0.000101,0.000122,...,0.000001,0.000003,1.234220e-07,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002
3,0.124375,0.142726,0.158633,0.166951,0.162531,0.142336,0.098433,0.000022,0.000101,0.000122,...,0.000001,0.000003,1.232996e-07,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002
4,0.124372,0.142726,0.158637,0.166958,0.162538,0.142339,0.098418,0.000022,0.000101,0.000122,...,0.000001,0.000003,1.231773e-07,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,0.047363,0.047739,0.046571,0.043514,0.038887,0.032830,0.021407,0.005802,0.011651,0.011211,...,0.001744,0.002313,3.400199e-04,0.003789,0.004532,0.004862,0.005131,0.005275,0.005168,0.003123
999996,0.047336,0.047706,0.046534,0.043475,0.038849,0.032796,0.021388,0.005805,0.011654,0.011214,...,0.001745,0.002314,3.404305e-04,0.003790,0.004533,0.004863,0.005133,0.005277,0.005170,0.003125
999997,0.047309,0.047673,0.046497,0.043436,0.038810,0.032761,0.021369,0.005808,0.011658,0.011218,...,0.001746,0.002316,3.408418e-04,0.003792,0.004535,0.004865,0.005135,0.005279,0.005172,0.003127
999998,0.047282,0.047640,0.046459,0.043396,0.038772,0.032727,0.021350,0.005810,0.011662,0.011221,...,0.001748,0.002317,3.412540e-04,0.003794,0.004537,0.004867,0.005137,0.005281,0.005173,0.003128


In [171]:
# import demesdraw
# import matplotlib.pyplot as plt

In [172]:
# graph = msprime.Demography.to_demes(demography)
# fig, ax = plt.subplots()  # use plt.rcParams["figure.figsize"]
# demesdraw.tubes(graph, ax=ax, seed=1)
# plt.show()

In [173]:
# ts.draw_svg(y_axis=True)

In [174]:
cut_ABC = t_1+t_2+cutpoints_ABC(n_int_ABC, 1/N_ABC)
cut_AB = t_1+cutpoints_AB(n_int_AB, t_2, 1/N_AB)
cut_AB_2 = np.array([t_1-t_m]+list(cut_AB)[1::])

left_lst = []
right_lst = []
tree_state = []
t_AB_vec = []
t_ABC_vec = []
for t in ts.trees():
    # Append start coordinate
    left_lst.append(t.interval.left)
    # Append end coordinate
    right_lst.append(t.interval.right-1)
    # Get all non-zero coalescent times
    ntimes = [ts.nodes()[n].time for n in t.nodes() if ts.nodes()[n].time not in [0, t_1-t_A, t_1-t_B, t_1-t_C]]
    ntimes = sorted(ntimes)
    # Get time of the first event
    mint = ntimes[0]
    mint2 = ntimes[1]
    # Find topology
    find_re = re.findall("n\d,n\d", t.as_newick(include_branch_lengths=False))[0]
    # Sort species within topology
    find_re = sorted(find_re.split(','))
    # If V0 or V1
    if find_re == ['n0', 'n1']:
        # If the time of the first coalescent is larger than the deepest speciation event
        if mint>=(t_1+t_2):
            state = (1, (mint>cut_ABC).sum()-1, (mint2>cut_ABC).sum()-1)
            # Append V1 state
        else:
            state = (0, (mint>cut_AB).sum()-1, (mint2>cut_ABC).sum()-1)
            # Append V0 state
    # If V2
    elif find_re == ['n0', 'n2']:
        state = (2, (mint>cut_ABC).sum()-1, (mint2>cut_ABC).sum()-1)
    # If V3
    elif find_re == ['n1', 'n2']:
        if mint>=(t_1+t_2):
            state = (3, (mint>cut_ABC).sum()-1, (mint2>cut_ABC).sum()-1)
        else:
            state = (4, (mint>cut_AB_2).sum()-1, (mint2>cut_ABC).sum()-1)
    else:
        state = (5, (mint>cut_ABC).sum()-1, (mint2>cut_ABC).sum()-1)
        print((find_re, (mint>cut_ABC).sum()-1, (mint2>cut_ABC).sum()-1))
    tree_state.append(state)
    t_AB_vec.append(mint)
    t_ABC_vec.append(mint2)

In [175]:
set(tree_state)

{(0, 0, 0),
 (0, 0, 1),
 (0, 0, 2),
 (0, 0, 3),
 (0, 0, 4),
 (0, 0, 5),
 (0, 0, 6),
 (1, 0, 0),
 (1, 0, 2),
 (1, 0, 5),
 (1, 0, 6),
 (1, 1, 1),
 (1, 1, 2),
 (1, 1, 3),
 (1, 1, 4),
 (1, 1, 5),
 (1, 1, 6),
 (1, 2, 5),
 (1, 2, 6),
 (1, 4, 4),
 (1, 4, 6),
 (1, 5, 5),
 (1, 5, 6),
 (1, 6, 6),
 (2, 0, 0),
 (2, 0, 1),
 (2, 0, 2),
 (2, 0, 3),
 (2, 0, 4),
 (2, 0, 5),
 (2, 0, 6),
 (2, 1, 2),
 (2, 1, 3),
 (2, 1, 4),
 (2, 1, 5),
 (2, 1, 6),
 (2, 2, 3),
 (2, 2, 4),
 (2, 2, 5),
 (2, 3, 3),
 (2, 3, 5),
 (2, 4, 5),
 (2, 4, 6),
 (3, 0, 0),
 (3, 0, 1),
 (3, 0, 2),
 (3, 0, 3),
 (3, 0, 4),
 (3, 0, 5),
 (3, 0, 6),
 (3, 1, 2),
 (3, 1, 4),
 (3, 1, 5),
 (3, 1, 6),
 (3, 2, 3),
 (3, 2, 4),
 (3, 2, 5),
 (3, 3, 4),
 (3, 3, 5),
 (3, 3, 6),
 (3, 4, 6),
 (4, 0, 0),
 (4, 0, 1),
 (4, 0, 2),
 (4, 0, 4),
 (4, 0, 5),
 (4, 0, 6)}

In [176]:
df = pd.DataFrame()
df['state'] = [str(i) for i in tree_state]
df['start'] = left_lst
df['end'] = right_lst
df

,state,start,end
0,"(0, 0, 5)",0.0,1497.0
1,"(0, 0, 5)",1498.0,3105.0
2,"(0, 0, 2)",3106.0,8300.0
3,"(0, 0, 5)",8301.0,10665.0
4,"(0, 0, 5)",10666.0,14327.0
...,...,...,...
892,"(3, 0, 6)",987168.0,991862.0
893,"(0, 0, 0)",991863.0,993045.0
894,"(0, 0, 0)",993046.0,998161.0
895,"(0, 0, 0)",998162.0,999065.0


In [177]:
df.to_csv(f'./tables/df_{n_int_ABC}.csv', index = False)
postprobs_df.to_csv(f'./tables/postprobs_df_{n_int_ABC}.csv', index = False)